In [1]:
import Snapshot_code
import pandas as pd

In [2]:
# new_df2 = pd.read_excel(r"C:\Users\NahianSiddique\OneDrive - Blend 360\Hilton\Analytical Projects\HGV 2022 VIP Analysis\Data\Model Sample\appended_data_18_19_20221213.xlsx")
new_df2 = pd.read_excel(r"C:\Users\NahianSiddique\OneDrive - Blend 360\Hilton\Analytical Projects\HGV 2022 VIP Analysis\Data\Model Sample\appended_data_21_22_20221213.xlsx")

new_df2 = new_df2.drop(
    columns=[
        "lead_id",
        "t0_baseline_date",
        "full_tour_id",
    ]
)

In [3]:
mapping_dict = {"Baseline": "dataset_1", "Segment_1": "dataset_2", "Segment_2": "dataset_3", "Segment_3": "dataset_4"}
# mapping_dict = {"Baseline": "dataset_1", "Segment_1": "dataset_2", "Segment_2": "dataset_3"}
# mapping_dict = {"Baseline": "dataset_1", "Segment_1": "dataset_2"}

# Set up variables for snapshot
file_name = "profiles_1-2-3_18-19"
seg_var = "source"
bin_vars_path = "Data/HGV_VIP_attributes_binning.csv"
# Read in file and set bins

In [4]:
new_df2["dream_Africa_ME"] = new_df2["dream_Africa_ME"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_Asia"] = new_df2["dream_Asia"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_Caribbean"] = new_df2["dream_Caribbean"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_Europe"] = new_df2["dream_Europe"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_HI"] = new_df2["dream_HI"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_Mexico"] = new_df2["dream_Mexico"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_South_America"] = new_df2["dream_South_America"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_USA"] = new_df2["dream_USA"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_Italy"] = new_df2["dream_Italy"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_FR"] = new_df2["dream_FR"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_South_Africa"] = new_df2["dream_South_Africa"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_Dubai"] = new_df2["dream_Dubai"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)
new_df2["dream_island"] = new_df2["dream_island"].apply(lambda x: 1 if x > 0 else 0 if x == 0 else x)

In [5]:
profile = Snapshot_code.Snapshot_Profile(
    profile_data=new_df2,
    segment_var=seg_var,
    continuous_path=bin_vars_path,
    segments=None,
    segment_names=None,
    include=None,
    variable_order=None,
    other_segment=False,
    file=file_name,
    exclude=[],
    PPT=True,
    plot_index=False,
    continuous=[],
    excludeother=False,
    mapping_dict=mapping_dict,
)

c:\Users\NahianSiddique\OneDrive - Blend 360\Documents\GitHub\Snapshot2.0\Snapshot_code.py:547: UserWarning: Incorrect length of names, replacing by segments
  warnings.warn("Incorrect length of names, replacing by segments")
c:\Users\NahianSiddique\OneDrive - Blend 360\Documents\GitHub\Snapshot2.0\explorer.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _sub_df[variable.name] = _sub_df[variable.name].fillna('NA')
c:\Users\NahianSiddique\OneDrive - Blend 360\Documents\GitHub\Snapshot2.0\explorer.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

KeyboardInterrupt: 